In [1]:
# libs
import os
import psycopg
import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
# credentials

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

TABLE_NAME = 'users_churn'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'churn_preprocessing_alexndem'
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = 'churn_model_alexndem'

In [5]:


with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(2) 

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,gender,streaming_movies,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,Female,No,0,Yes,No,None,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,Male,No,0,No,No,No,0


In [6]:
id_columns = ['id', 'customer_id']

### Разбивка колонок по типам

In [7]:
obj_df = df.drop(columns=id_columns).select_dtypes(include="object")

In [8]:
obj_df.head(2)

,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,gender,streaming_movies,partner,dependents,multiple_lines
0,Month-to-month,Yes,Electronic check,DSL,No,Yes,No,No,No,Female,No,Yes,No,None
1,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,Male,No,No,No,No


In [9]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(obj_df[cat_columns])

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out())

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,gender,...,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,internet_service_None,gender_Male
0,Month-to-month,Yes,Electronic check,DSL,No,Yes,No,No,No,Female,...,No,None,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,Male,...,No,No,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [10]:
num_columns = ["monthly_charges", "total_charges"]
df[num_columns] = df[num_columns].fillna(0)
num_df = df[num_columns]


n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
# get all columns after the intercept and original features
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,...,total_charges_robust,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
0,29.85,29.85,0.014583,0.335266,0.554993,0.095040,0.000118,0.0,0.040533,0.454885,...,-0.402798,891.0225,891.0225,8.910225e+02,26597.021625,2.659702e+04,2.659702e+04,2.659702e+04,0.0,0.0
1,56.95,1889.50,0.000116,0.094742,0.554677,0.335807,0.014658,0.0,0.004242,0.228653,...,0.146087,3243.3025,107607.0250,3.570210e+06,184706.077375,6.128220e+06,2.033235e+08,6.745912e+09,1.0,1.0


In [11]:
num_df.columns

Index(['monthly_charges', 'total_charges', 'monthly_charges_sp_0',
       'monthly_charges_sp_1', 'monthly_charges_sp_2', 'monthly_charges_sp_3',
       'monthly_charges_sp_4', 'monthly_charges_sp_5', 'total_charges_sp_0',
       'total_charges_sp_1', 'total_charges_sp_2', 'total_charges_sp_3',
       'total_charges_sp_4', 'total_charges_sp_5', 'monthly_charges_q_100',
       'total_charges_q_100', 'monthly_charges_robust', 'total_charges_robust',
       'monthly_charges^2', 'monthly_charges total_charges', 'total_charges^2',
       'monthly_charges^3', 'monthly_charges^2 total_charges',
       'monthly_charges total_charges^2', 'total_charges^3',
       'monthly_charges_bin', 'total_charges_bin'],
      dtype='object')

In [12]:
encoder_pol.get_feature_names_out(num_columns)

array(['1', 'monthly_charges', 'total_charges', 'monthly_charges^2',
       'monthly_charges total_charges', 'total_charges^2',
       'monthly_charges^3', 'monthly_charges^2 total_charges',
       'monthly_charges total_charges^2', 'total_charges^3'], dtype=object)

In [13]:
numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns), ('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)])

categorical_transformer = ColumnTransformer(transformers=[('encoder', encoder_oh, cat_columns)])

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df, transformed_df], axis=1)
df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,num__kbd__monthly_charges,num__kbd__total_charges,cat__encoder__type_One year,cat__encoder__type_Two year,cat__encoder__payment_method_Credit card (automatic),cat__encoder__payment_method_Electronic check,cat__encoder__payment_method_Mailed check,cat__encoder__internet_service_Fiber optic,cat__encoder__internet_service_None,cat__encoder__gender_Male
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [15]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2025/06/03 16:30:48 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [16]:
run_id

'01e47211b28c4a6cbc96fc7f9302b453'

### Обучение модели

In [17]:
from sklearn.metrics import mean_absolute_error, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, confusion_matrix, precision_score, recall_score, f1_score, log_loss

from catboost import CatBoostClassifier

In [18]:
cat_features = df.drop(columns=['end_date']).dtypes[df.dtypes == 'object'].index.tolist()
drop_features = df.dtypes[df.dtypes == 'datetime64[ns]'].index.tolist() + ['end_date']
target = ['target']

df['total_charges'] = df['total_charges'].fillna(0)
df.fillna(0, inplace=True)

/tmp/ipykernel_3182/451513483.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0, inplace=True)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=target + drop_features), df[target], test_size=0.2, random_state=42)

model = CatBoostClassifier(iterations=300, cat_features=cat_features, verbose=False)
model.fit(X_train, y_train)
proba = model.predict_proba(X_test)[:, 1]
prediction = model.predict(X_test)

In [20]:
_, err1, _, err2 = confusion_matrix(y_test, prediction).ravel()
auc = roc_auc_score(y_test, proba)
logloss = log_loss(y_test, proba)

In [24]:
EXPERIMENT_NAME = "churn_task_alexdem"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_alexdem_b2c"

pip_requirements= "./requirements.txt"
signature = mlflow.models.infer_signature(X_test.values, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    model_info = mlflow.catboost.log_model( 
			cb_model=model,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME,
            pip_requirements=pip_requirements,
            signature=signature,
            input_example=input_example,
            metadata=metadata,
            await_registration_for=60
		)
    mlflow.log_metrics({'auc': auc, 'logloss': logloss, 'err1': err1, 'err2': err2})

/home/mle-user/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'churn_model_alexdem_b2c' already exists. Creating a new version of this model...
2025/06/03 16:36:13 INFO mlflow.tracking._

In [25]:
run_id

'8bc0cb90e532407a9c8b76975d8edf8c'